In [1]:
"""
This code makes diagnostic plots for raw and postprocessed WRF-Solar output
Postprocessing using the Kalman Filter is from Rafael Alvarenga's code (rafael.alvarenga@etu.univ-guyane.fr)
"""
from file_func import *
from plotting import *
import matplotlib.image as mpimg
import seaborn as sns

In [2]:
li = pd.DataFrame()
method= 'KF'
nb_historical_days = nbhd
for ens in ['ens0','ens1','ens2','ens3','ensmean']:
    for domain in ['d01', 'd02']:
        dir_path_recursive = f'{main_dir_path_recursive}/{ens}/{domain}/{station_name}/{nb_historical_days}_day_{method}'
        all_files = glob.glob(dir_path_recursive + "/*_df.csv")

        for filename in all_files:
            df = pd.read_csv(filename)
            df['rmse_mod'] = fitting(df)[7]
            df['r_mod'] = fitting(df)[0]
            
            df['kfp_rmse'] = fitting_pred(df)[7]
            df['kfp_r'] = fitting_pred(df)[0]
            
            df['kf_rmse'] = kf_fitting_pred(df)[7]
            df['kf_r'] = kf_fitting_pred(df)[0]
   
            li = li.append(df)
            print(filename)


output_junaug/ens2/d01/MO/42_day_KF/ens2_d01_42_day_KF_df.csv
output_junaug/ensmean/d01/MO/42_day_KF/ensmean_d01_42_day_KF_df.csv


In [3]:
for domain in ['d01', 'd02']:
    ens0 = li[(li['ens'] == 'ens0') & (li['domain'] == domain)]
    ens0['ghi_mod0'] = ens0['ghi_mod']
    ens0 = ens0[['Time','sza', 'flag_clear', 'ghi_obs', 'ghi_mod0']]

    ens1 = li[(li['ens'] == 'ens1') & (li['domain'] == domain)]
    ens1['ghi_mod1'] = ens1['ghi_mod']
    ens1 = ens1[['Time','ghi_mod1']]

    ens2 = li[(li['ens'] == 'ens2')  & (li['domain'] == domain) ]
    ens2['ghi_mod2'] = ens2['ghi_mod']
    ens2 = ens2[['Time','ghi_mod2']]

    x = ens0.merge(ens1,how='inner',on="Time")
    x = ens2.merge(x,how='inner',on="Time")
    x['ghi_mod'] = x[['ghi_mod0', 'ghi_mod1', 'ghi_mod2']].mean(axis=1)
    if domain == 'd01':
        x5_li = x
        x5_li['rmse_mod'] = fitting(x)[7]
        x5_li['r_mod'] = fitting(x)[0]
    if domain == 'd02':
        x1_li = x
        x1_li['rmse_mod'] = fitting(x)[7]
        x1_li['r_mod'] = fitting(x)[0]
        
    x['dw_solar'] = x['ghi_obs']
    x['G_nam'] = x['ghi_mod']
    x['zen'] = x['sza']
    x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
    
    
    
    x.to_csv(f'{main_dir_path_recursive}/oldensmean_{domain}_model.csv')

/var/folders/m6/4qg_57053p93nhjn29vdbv940000gp/T/ipykernel_12169/2133110398.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens2['ghi_mod2'] = ens2['ghi_mod']


ValueError: Inputs must not be empty.

In [12]:
r_input = li[['Time', 'sza', 'ghi_obs', 'LT','ghi_mod', 'GHI_pred_kf_only', 'GHI_pred_improved_fin', 'ens', 'domain', 'flag_clear']]
for ens in ['ens2']:
    for domain in ['d01']:
        print('start')
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain) ]
        nb_LTs = len(np.unique(x.LT))
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x = x[x['flag_clear'] == 'N']
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['ghi_mod']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_model_N.csv')
        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        nb_LTs = len(np.unique(x.LT))
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x = x[x['flag_clear'] == 'N'] 
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['GHI_pred_kf_only']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_kf_N.csv')
        #####################################################################        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        print(x)
        nb_LTs = len(np.unique(x.LT))
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x = x[x['flag_clear'] == 'Y'] 
        print(x)
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['ghi_mod']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_model_Y.csv')
        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        nb_LTs = len(np.unique(x.LT))
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x = x[x['flag_clear'] == 'Y'] 
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['GHI_pred_kf_only']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        print(x)
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_kf_Y.csv')
        #####################################################################
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        nb_LTs = len(np.unique(x.LT))
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['ghi_mod']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_model.csv')
        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        x = x.iloc[nb_LTs*nb_historical_days:,:] 
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['GHI_pred_kf_only']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_kf.csv')
        print('end')
        
   

start
                           Time        sza    ghi_obs    LT     ghi_mod  \
0     2020-06-02 05:30:00+08:00  89.364776  23.498014   330   44.672157   
1     2020-06-02 05:40:00+08:00  87.329351  23.694669   340   59.562877   
2     2020-06-02 05:50:00+08:00  85.172973  45.740527   350   74.453596   
3     2020-06-02 06:00:00+08:00  82.966751  83.262937   360   89.344315   
4     2020-06-02 06:10:00+08:00  80.734649  96.432498   370  127.934884   
...                         ...        ...        ...   ...         ...   
6448  2020-08-24 17:30:00+08:00  80.520002  13.031749  1050   75.409211   
6449  2020-08-24 17:40:00+08:00  82.881484  12.623990  1060   53.310402   
6450  2020-08-24 17:50:00+08:00  85.219265   5.284972  1070   31.211592   
6451  2020-08-24 18:00:00+08:00  87.504768   2.643337  1080    9.112782   
6452  2020-08-24 18:10:00+08:00  89.647165   1.630319  1090    7.593985   

      GHI_pred_kf_only  GHI_pred_improved_fin   ens domain flag_clear  
0            44.67215

In [5]:
li.columns

Index(['Unnamed: 0', 'Time', 'CMP22_Total_Solar', 'SPN1_Total_Solar',
       'SPN1_Diff_Solar', 'CGR4_IR', 'dhi', 'ghi_a', 'sza', 'cossza', 'dni',
       'MM', 'DD', 'HH', 'mm', 'GHI_in', 'DNI_in', 'DHI_in', 'cossza_b',
       'SPN1_Total_Solar_N', 't2_lim', 'cossza_noon', 'FT_t', 'FT_TOA',
       'FT_TOA_t', 't3_llim', 't3_ulim', 'Diffuse_Ratio', 'SPN1_Diff_Solar_N',
       'sigma', 'ghi_cc_val', 'dhi_cc_val', 't1_lim', 'flag_clear', 'ghi_obs',
       'ens', 'domain', 'station_name', 'ghi_mod', 'YY', 'Error_rel',
       'Kc_GHI_pred', 'Kc_GHI_obs', 'Kc_obs_bias', 'LT', 'GHI_pred_kf_only',
       'GHI_pred_improved_fin', 'rmse_mod', 'r_mod', 'kfp_rmse', 'kfp_r',
       'kf_rmse', 'kf_r'],
      dtype='object')

In [ ]:
r_input = li[['Time', 'sza', 'ghi_obs', 'ghi_mod', 'GHI_pred_kf_only', 'GHI_pred_improved_fin', 'ens', 'domain', 'flag_clear']]
for ens in ['ens0','ens1','ens2','ens3','ensmean']:
    for domain in ['d01', 'd02']:
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['ghi_mod']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_model.csv')
        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['GHI_pred_kf_only']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_kf.csv')
        
        x = r_input[(r_input['ens'] == ens) & (r_input['domain'] == domain)]
        x['dw_solar'] = x['ghi_obs']
        x['G_nam'] = x['GHI_pred_improved_fin']
        x['zen'] = x['sza']
        x = x[['Time', 'zen', 'dw_solar', 'G_nam']]
        x.to_csv(f'{main_dir_path_recursive}/{ens}_{domain}_kfp.csv')